In [12]:
import json
import jieba
import re
import numpy as np
import pandas as pd

sample_20220821 = pd.read_csv('./sample_20220821_spark.csv')

def tags_jieba_fun(series):
    tags = series["tags"]

    if tags.startswith("[") and tags.endswith("]"):
        tags = json.loads(tags)
    else:
        tags = re.split(",|，", tags)
    
    tags_str = " ".join(tags)
    tags_list = jieba.lcut(tags_str, cut_all=False)
    return " ".join(tags_list)

df1 = sample_20220821
# 空值填充
df1['tags'].fillna('[]', inplace=True)
df1['tags_jieba'] = df1.apply(tags_jieba_fun, axis=1)
df1


,Unnamed: 0,cv_id,jd_id,tags,skills,tags_jieba
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,"[""市场策划"",""管理"",""推广"",""营销"",""品牌推广""]","[""Ai"",""Ar"",""App"",""优化"",""营销"",""策划"",""运营"",""搭建"",""电商""...",市场策划 管理 推广 营销 品牌 推广
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,"[""销售"",""团队管理"",""市场推广"",""AI"",""负责管理""]","[""it规划咨询"",""云计算咨询""]",销售 团队 管理 市场推广 AI 负责管理
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,"[""投资"",""临床"",""生物"",""生物医药""]","[""VC"",""PPT"",""SQL"",""VBA"",""Kpi"",""Excel"",""C/C++"",...",投资 临床 生物 生物医药
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,"[""运营"",""产品运营"",""内容运营"",""美工"",""互联网产品""]","[""Vi"",""Pr"",""App"",""Crm"",""优化"",""策划"",""营销"",""搭建"",""融资...",运营 产品 运营 内容 运营 美工 互联网 产品
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,"[""架构"",""数字化"",""管理"",""云计算"",""区块链""]","[""PPT"",""Word"",""Excel"",""Visio"",""营销"",""运营"",""搭建"",""...",架构 数字化 管理 云 计算 区块 链
...,...,...,...,...,...,...
81291,81291,8a69f60582b081f50182b92ebdab1e84,8a69f62f82926a860182a0d7cd7d28bb,"[""仓储"",""物流"",""管理"",""沟通能力"",""软件""]","O2O,Erp,Wms,Kpi,优化,运营,搭建,驾驶,电商,性能,架构,招投标,施工图,绩...",仓储 物流 管理 沟通 能力 软件
81292,81292,8a69f61382b080760182b2fb552a0978,8a69f62f82926a860182af1bf161226d,"[""Photoshop"",""Coredraw"",""AI"",""3D工业软件"",""软件""]","Zcool,电商,优化,搭建,装修,页面设计,平面设计",Photoshop Coredraw AI 3D 工业 软件 软件
81293,81293,8a69f68b828a2cd901828f891bd10be1,8a69c4d6826e58640182804931425820,"[""游戏"",""动画设计"",""审核"",""美术"",""软件""]","C1,C4D,Icon,舞蹈,美术,项目经理,视频制作,沟通能力",游戏 动画设计 审核 美术 软件
81294,81294,8a69f68b828a2cd901828fc2ff6a4885,8a69c4d6826e5864018285e46d3f17d7,"[""沟通能力"",""归档"",""法规"",""评审"",""产品开发""]","制药,内科,配置,策划,美容,团队销售,生物医药",沟通 能力 归档 法规 评审 产品开发


In [22]:
from zhconv import convert

def tags_jieba_filter_fun(series):

    tgas_list_filter = []
    # 得到tag列表
    tags_list = series["tags_jieba"].split(" ")
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in tags_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            tgas_list_filter.append(tag)
    return " ".join(tgas_list_filter)
    
df1['tags_jieba_filter'] = df1.apply(tags_jieba_filter_fun, axis=1)
df1


,Unnamed: 0,cv_id,jd_id,tags,skills,tags_jieba,tags_jieba_filter
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,"[""市场策划"",""管理"",""推广"",""营销"",""品牌推广""]","[""Ai"",""Ar"",""App"",""优化"",""营销"",""策划"",""运营"",""搭建"",""电商""...",市场策划 管理 推广 营销 品牌 推广,市场策划 管理 推广 营销 品牌 推广
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,"[""销售"",""团队管理"",""市场推广"",""AI"",""负责管理""]","[""it规划咨询"",""云计算咨询""]",销售 团队 管理 市场推广 AI 负责管理,销售 团队 管理 市场推广 ai 负责管理
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,"[""投资"",""临床"",""生物"",""生物医药""]","[""VC"",""PPT"",""SQL"",""VBA"",""Kpi"",""Excel"",""C/C++"",...",投资 临床 生物 生物医药,投资 临床 生物 生物医药
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,"[""运营"",""产品运营"",""内容运营"",""美工"",""互联网产品""]","[""Vi"",""Pr"",""App"",""Crm"",""优化"",""策划"",""营销"",""搭建"",""融资...",运营 产品 运营 内容 运营 美工 互联网 产品,运营 产品 运营 内容 运营 美工 互联网 产品
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,"[""架构"",""数字化"",""管理"",""云计算"",""区块链""]","[""PPT"",""Word"",""Excel"",""Visio"",""营销"",""运营"",""搭建"",""...",架构 数字化 管理 云 计算 区块 链,架构 数字化 管理 计算 区块
...,...,...,...,...,...,...,...
81291,81291,8a69f60582b081f50182b92ebdab1e84,8a69f62f82926a860182a0d7cd7d28bb,"[""仓储"",""物流"",""管理"",""沟通能力"",""软件""]","O2O,Erp,Wms,Kpi,优化,运营,搭建,驾驶,电商,性能,架构,招投标,施工图,绩...",仓储 物流 管理 沟通 能力 软件,仓储 物流 管理 沟通 能力 软件
81292,81292,8a69f61382b080760182b2fb552a0978,8a69f62f82926a860182af1bf161226d,"[""Photoshop"",""Coredraw"",""AI"",""3D工业软件"",""软件""]","Zcool,电商,优化,搭建,装修,页面设计,平面设计",Photoshop Coredraw AI 3D 工业 软件 软件,photoshop coredraw ai 3d 工业 软件 软件
81293,81293,8a69f68b828a2cd901828f891bd10be1,8a69c4d6826e58640182804931425820,"[""游戏"",""动画设计"",""审核"",""美术"",""软件""]","C1,C4D,Icon,舞蹈,美术,项目经理,视频制作,沟通能力",游戏 动画设计 审核 美术 软件,游戏 动画设计 审核 美术 软件
81294,81294,8a69f68b828a2cd901828fc2ff6a4885,8a69c4d6826e5864018285e46d3f17d7,"[""沟通能力"",""归档"",""法规"",""评审"",""产品开发""]","制药,内科,配置,策划,美容,团队销售,生物医药",沟通 能力 归档 法规 评审 产品开发,沟通 能力 归档 法规 评审 产品开发


In [23]:
df1['tags_jieba_filter']

# 将新生成的列tags_jie保存下来。可以发现需要进一步过滤。
with open('./tags_jieba_filter_temp', "w") as fo:
    for line in df1['tags_jieba_filter']:
        fo.write(line + '\n')


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 文件列表
# text = ["The quick brown fox jumped over the lazy dog.",
#         "The dog.",
#         "The fox"]
text = df1['tags_jieba_filter']
# 建立transform
vectorizer = TfidfVectorizer()
# 分词，建立词汇表
vectorizer.fit(text)

# 输出结果
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)

with open("./word_idf_temp", "w") as fo:
    for word, index in vectorizer.vocabulary_.items():
        fo.write("{}\t{}\n".format(word, vectorizer.idf_[index]))
        #print(word, vectorizer.idf_[index])
    

# 词典的大小
print(len(vectorizer.vocabulary_))
# 编码文件
vector = vectorizer.transform(text)
# 输出编码结果
print(vector.shape)
print(vector.toarray())


2393
(81296, 2393)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [26]:
print(type(vector.toarray()))
tfidf = pd.DataFrame(vector.toarray())
tfidf

<class 'numpy.ndarray'>


,0,1,2,3,4,5,6,7,8,9,...,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.402959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
tfidf['sum'] = tfidf.apply(sum, axis=1)
tfidf

,0,1,2,3,4,5,6,7,8,9,...,2384,2385,2386,2387,2388,2389,2390,2391,2392,sum
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.993638
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.522898
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.923468
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.343408
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.166106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.316813
81292,0.0,0.0,0.0,0.0,0.402959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.799790
81293,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.924344
81294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.665284


In [33]:
for index, value in enumerate(tfidf.iloc[1]):
    if value:
        print(index, value)


23 0.3980843066467165
1053 0.3160548767987825
1236 0.48180611834061005
1907 0.16419906821926017
2159 0.6460554138718986
2309 0.2552491570712822
2393 4.5228978818971


In [34]:
# 市场策划 管理 推广 营销 品牌
# 销售 团队 管理 市场推广 ai 负责管理
words = ['市场策划', '管理', '推广', '营销', '品牌']
words = '销售 团队 管理 市场推广 ai 负责管理'.split(" ")
for word in words:
    print(vectorizer.vocabulary_[word])

2309
1053
1907
1236
23
2159


In [ ]:
'''
接下来工作
1.切词后做一些过滤操作，比如去除数字、单个字符、标点等
2.降维
'''